In [22]:
from broadlink import remote
import statistics as st



In [23]:

def _threshold(data):
    """Return a good short/long threshold for Broadlink pulses."""
    data = sorted(data)
    if len(data) < 4:                      # too few samples → fallback
        return st.median(data)

    # two -cluster split: lower ⅓ vs upper ⅔
    q1, q2 = st.quantiles(data, n=3)
    return (q1 + q2) / 2                  # mid-point between the clusters
def manchester_decode(durations, gap=2000):
    """Return (bitstring, start_index) from a raw Broadlink pulse list."""
    DATA = [t for t in durations if t < gap]          # 1. cut inter-frame gaps
    if len(DATA) < 4:
        raise ValueError("Not enough data after gap-filtering")

    THR     = _threshold(DATA)                        # 2. quantise
    symbols = ['S' if t < THR else 'L' for t in DATA]

    # 3. auto-resync
    bits, i = [], 0
    while i + 1 < len(symbols):
        a, b = symbols[i], symbols[i + 1]

        if a == b:            # LL or SS → still in leader or a timing glitch
            i += 1            # shift by one symbol and try again
            continue

        bits.append('0' if (a, b) == ('S', 'L') else '1')
        i += 2

    return ''.join(bits), i

In [21]:
bits, start = manchester_decode(remote.data_to_pulses(raw))
bits[0:56+4]

'101011000100001001100000001010110001000010011000000010101100'